In [ ]:
import numpy as np
import sys
import warnings
import plotly.express as px 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import plotly.express as px 
sys.path.insert(1, "../src/utils/")
from agent import Environment
sys.path.insert(1, "plotly_graph/")
from functions4tuto import plotly_trajectory, control_fall_simulation, booster_reward, plotly_all_reward
JOSN_file = "params/rocket_tuto_1.json"

# <span style="color:orange">Input file</span>

Before to start trainning we need to define our environment. In this example, the system is a rocket. 
It falls from a starting point and the goal is to reach a target with a minimum speed and maximize fuel.
Variable's names, their initial values and boundaries limit are defined into a JSON file (or can be given directly into a dictionary)

## <span style="color:orange">Variables</span>

Variables can be categorize into 3 classes:
* `states_variables` : variable used as coordinate to describe our system
* `agent_variables` : variable use as agent. Their values are changed for each iteration
* 3th category are other variables. They are not used to describe our environment but they can be usefull to monitor information or to compute intermediate value. 
  There is no key for this kind of variable. Consider them as variables present into `initial_values` field and that are not `states_variables` and `agent_variables`

You can access to the name of state and agent variables, with the attibute `states_variables` and `agent_variables`.

    "states_variables" : ["pos_y", "acceleration_y", "speed_y"],
    "agent_variables" : ["booster"]

## <span style="color:orange">Initial system</span>

After to name `states_variables` and `agent_variables`, next step is to define initial state.
It will be use as environment coordinates at the beginning of each episode. Initially, it comprises 
the values of state_variables, followed by agent_variables, and finally other variables that are 
not used for the system's coordinates.

    "initial_values" : {
      "pos_y" : [175.0],
      "acceleration_y": [0.0],
      "speed_y": [0.0],
      "angle" : [0.0],
      "booster" : [0.0],
      "alpha" : [0.0],
      "futur_pos_y" : [175.0],
      "m_fuel" : [100],
      "weight_rocket" : [105],
      "weight_dry_rocket" : [5],
      "G" : [1.62],
      "m_fuel_ini" : [100.0],
      "pos_y_star": [0.0]
      }


## <span style="color:orange">Limit</span>

Q-learning algorithms model events as a Markov process. Therefore, it is necessary to discretize our environment space. 
We define lower and upper bounds, as well as the number of divisions we want to use to discretize the variable space.


    "limit" : {
      "pos_y" : [0.0, 300.0, 61],
      "acceleration_y": [-20.0, 20.0, 21],
      "speed_y": [-50.0, 50.0, 21],
      "angle" : [-0.8, 0.8, 17],
      "booster" : [0.0, 1.0, 3],
      "alpha" : [-0.1, 0.1, 3],
      "m_fuel" : [0.0, 100, 101]
    }


## <span style="color:orange">Agents's actions</span>
    "n_action" : {
      "booster": {"0" : 0.0, "1" : 0.5, "2" : 1.0}
    }

After defining the variables and their initial values, we proceed to define actions that apply to the agent variables. In this example, we have 1 agent that can take 3 actions:

For the booster:
  * "0": Booster is off.
  * "1": Booster is turned on to half of its power.
  * "2": Booster is turned on to its full power.

## <span style="color:orange">Actions to take</span>

    "action_to_take" : {
      "booster": {"$booster$" : "$action}
    }

Actions change the agent variables by modifying their values based on the action taken, which are retrieved from the n_action dictionary.

## <span style="color:orange">System's evolution and reward</span>

Last fields are how variables evolve after agents's action and how reward are computed.
Only variables present in field inital value are stored. Other variables present are just temporary and are lost after each iteration.
Reward values are stored into a dictionnary, the keys are agent variable's name. 

### <span style="color:orange">Equation variables</span>

    "equations_variables": {
        "$F$" : "600",
        "$m_fuel$" : "$m_fuel$ - $booster$ *10 -$angle$ *10",
        "$weight_rocket$" : "$weight_dry_rocket$ + $m_fuel$",
        "dt" : "0.5",
        "$theta$" : "0.0",
        "$y_0$" : "$pos_y$",
        "$Vy_0$" : "$speed_y$",
        "$angle$" : "$theta$ + $alpha$",
        "$acceleration_y$" : "($F$/(5+$weight_rocket$) * np.cos($angle$)) * $booster$ - $G$",
        "$speed_y$" : "($F$/(5+$weight_rocket$) * np.cos($angle$)) * $booster$ * $dt$ - $G$ * $dt$ + $Vy_0$",
        "$pos_y$": "(0.5 * $F$/(5+$weight_rocket$) * np.cos($angle$)) * $booster$ * $dt$**2 - $G$ * $dt$**2 + $Vy_0$ * $dt$ + $y_0$",
        "$futur_pos_y$" : "$pos_y$ + 3 * $speed_y$"
    },

### <span style="color:orange">Reward</span>

The reward indicates the immediate benefit or cost associated with the action.
The scalar feedback signal that the environment sends to the agent after it takes an action are defined into this dictionnary.

    "equations_rewards": {
      "$booster$" : "-($pos_y$ - $pos_y_star$)**2 + $m_fuel$/$m_fuel_ini$"
    }

### <span style="color:orange">stop_episode</span>

The optional `stop_episode` field stops episode when system reaches a desired goal.
You can give on or many features and specify stop condition.
If feature has 1 value, the value must be equal. In other hand, if feature has 2 values [min_limit, max_limit]. Criterion is bounded feature >= min_limit and feature <= max_limit.


    "stop_episode" : {
      "pos_y" : [1, 0],
      "acceleration_y" : [-2,2],
      "speed_y" : [-1,1]
  }

# <span style="color:orange">Lets see how system evolves</span>

A premilary work must be done on reward function. Indeed, we have 2 agents. Each one has an impact on rocket trajectory. 

Lets simulate a simple case. Our rocket start with no speed. The only force applied on it is G (Gravitational constant). 
The rocket falls down straight on the planetoid (angle is zero no needs to correct it) and without friction. The goal is to reach a point (pos_x_star, pos_y_star).

We start booster engine when we are close to the ground. For that, we compute the new position after 3 * dt (`futur_pos_y`).
When `futur_pos_y` is bellow zero, the rocket activates engine to compensate the fall speed and avoid the crash. 

In [ ]:
# Create an environment object with the rules defined previously
env = Environment(JOSN_file, check_model = True)

In [ ]:
# Create an environment object with the rules defined previously
env = Environment(JOSN_file, check_model = False)
flag = "0"
flag_to_continue = True
# monitor action takes for each iteration
actions = {"action_booster" : []} 
while flag_to_continue:    
    _, rewards, done, problem, info = env.step([flag])
    actions["action_booster"].append(flag)
    if env.futur_pos_y[-1] <= 0 and env.m_fuel[-1] > 0:
        flag = "2"
    # stop engine if there is no fuel
    if env.m_fuel[-1] == 0:
        flag = "0"
    # stop simulation
    if env.pos_y[-1] < 0:
        flag_to_continue = False
        # delete last state because rocket is bellow to the ground
        env.delete_last_states()

Each state of our system is saved in `env`. We can access the last state of our system using the method `last_state()`.
If we want to access a specific range of states, it is the method `last_state()` (by default, all states are loaded).

Lets visualize how the system evolves !

In [ ]:
# We save result into a pandas data frame and add column for time
df_traj = pd.concat([pd.DataFrame(env.all_states()), pd.DataFrame(actions)], axis = 1)
df_traj["iter"] = np.arange(0, df_traj.shape[0])

In [ ]:
plotly_trajectory(df_traj)

At t=0, the rocket is held in the air and its velocity is 0. Once released, the only force acting on the rocket is gravity. 
Therefore, our rocket will accelerate constantly and gain speed. At t=24, the projection of the rocket's position is below 0. 
This means that the rocket will reach the ground in 3 time steps if nothing is done. To avoid catastrophe, the boosters are activated. 
The boosters will provide acceleration that counteracts the gravitational constant. 
Therefore, as the rocket burn fuel its weight decreases and the acceleration increases and the rocket gains speed in the opposite direction of its fall.

# <span style="color:orange">Design reward functions</span>

Let's visualize the evolution of the reward functions by using the attribut `reward`. As a reminder, the reward function for the booster corresponds to the vertical distance relative to pos_y_star plus the fuel ratio.
While for alpha (which corresponds to the angle of the rocket), its reward function corresponds to the horizontal distance relative to pos_x_star, minus the rocket's tilt angle.

In [ ]:
fig = px.line(pd.DataFrame(env.rewards),
        labels=dict(index="time", value="Reward", variable="Agent") )
fig.update_layout(height=400, width=600, title_text="Agent's reward")

## <span style="color:orange">Constraint acceleration and speed</span>


The current reward function for the booster is far from optimal. As evident from the data, the value increases steadily and peaks at the 26th iteration. However, the engine won't start until then, risking a crash. Instead of using `pos_y`, we can utilize `future_pos_y` to allow ample time to start the engine and prevent a collision. While predicting the next position after a 3-time step works well in this example, in other scenarios, the rocket's speed may be excessive, rendering 3 time steps insufficient. To mitigate such scenarios, we could also impose constraints on the vehicle's acceleration and speed.

We can modulate speed in a way, system will be high penalize when it begans to be out of bounds:

$$  \exp(1) - \exp(Max( \frac{|speed\_y|}{speed\_y\_limit} , 1)) $$ 



The same reasoning can be applied to acceleration.

$$  \exp(1) - exp(Max( \frac{|acceleration\_y|}{acceleration\_y\_limit} , 1)) $$ 

In [ ]:
speed_y_limit = 10
speed_y = np.arange(-15,16)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = speed_y,
    y = np.array([ np.exp(1) - np.exp(np.max([val, 1])) for val in np.abs(speed_y)/speed_y_limit ]),
    name='Speed constraint',
    mode='lines+markers'
))
acceleration_y_limit = 5
acceleration_y = np.arange(-11,12)
fig.add_trace(go.Scatter(
    x = acceleration_y,
    y = np.array([ np.exp(1) - np.exp(np.max([val, 1])) for val in np.abs(acceleration_y)/acceleration_y_limit ]),
    name='Acceleration constraint',
    mode='lines+markers'
))

fig.update_xaxes(title_text = "Speed or acceleration")
fig.update_yaxes(title_text = "Penalty")
fig.update_layout(height=400, width=600, title_text="Function to penalize speed")

## <span style="color:orange">Height boundaries for furtur pos y</span>


After we've imposed constraints on acceleration and speed, the final step is to discourage the system from crashing. 
To achieve this, we set a minimum bound for the height.

$$ -\exp(0) + \exp(Min( y\_lower\_limit - futur\_pos\_y, 0)) $$

In [ ]:
y_lower_limit = 0
futur_pos_y = np.arange(-7, 7)


fig = go.Figure()
fig.add_trace(go.Scatter(
    x = futur_pos_y,
    y = np.array([-np.exp(0) + np.exp(np.min([val, 0])) for val in futur_pos_y - y_lower_limit ]),
    name='height constraint',
    mode='lines+markers'
))

fig.update_xaxes(title_text = "height")
fig.update_yaxes(title_text = "Penalty")
fig.update_layout(height=400, width=600, title_text="Function to penalize speed")

We can also discourage the system from going too far away. 
To achieve this, we set a minimum bound for the height.

$$ \exp(0) + \exp(Min( y\_upper\_limit - futur\_pos\_y, 0)) $$

In [ ]:
y_upper_limit = 15
futur_pos_y = np.arange(7, 20)


fig = go.Figure()
fig.add_trace(go.Scatter(
    x = futur_pos_y,
    y = np.array([np.exp(0) + np.exp(np.min([val, 0])) for val in -futur_pos_y + y_upper_limit ]),
    name='height constraint',
    mode='lines+markers'
))

fig.update_xaxes(title_text = "height")
fig.update_yaxes(title_text = "Penalty")
fig.update_layout(height=400, width=600, title_text="Function to penalize speed")

By combining the 2 constraint we got:

$$ \exp(Min( y\_upper\_limit - futur\_pos\_y, 0)) + \exp(Min( y\_lower\_limit - futur\_pos\_y, 0)) -2 $$

In [ ]:
y_upper_limit = 15
y_lower_limit = 0

futur_pos_y = np.arange(-7, 23)


fig = go.Figure()
fig.add_trace(go.Scatter(
    x = futur_pos_y,
    y = np.array([-2 + np.exp(np.min([val1, 0])) + np.exp(np.min([val2, 0])) for val1, val2 in zip(futur_pos_y - y_lower_limit, -futur_pos_y + y_upper_limit) ]),
    name='height constraint',
    mode='lines+markers'
))

fig.update_xaxes(title_text = "height")
fig.update_yaxes(title_text = "Penalty")
fig.update_layout(height=400, width=600, title_text="Function to counstraint height exploration")

# <span style="color:orange">Evaluate penalties function</span>

We designed our function to penalize undesired behavior. It's time to simulate what will happen. 
The system begins from the same starting point, and we add a new condition: if the speed or acceleration is above or below their respective upper and lower limits, the rocket's engine is turned off or on, depending on the situation.

In [ ]:
dt, df_penalty = control_fall_simulation(JOSN_file, 
                            acceleration_y_constraint = 5, 
                            speed_y_limit = 17, 
                            y_lower_limit = 0)
# Trajecotry result do not plot event after the last propulsion
plotly_all_reward(dt, df_penalty)

# <span style="color:orange">Conclusion</span>

We designed each component of our reward function. Initially, the primary component is the distance penalty. As the rocket approaches its endpoint, this penalty decreases further. Similarly, for penalty rewards, when they are out of bounds, they are heavily penalized. The last time the booster was turned on, the acceleration was out of bounds, and the system was heavily penalized; moreover, its speed was too fast at the end. If we only consider the first 70 time steps, our reward function seems good.

The final step consists of assigning weights to our components to prevent a value of 0 at the beginning and to give more weight to the distance component and the y_constraint.

$$ reward =  3 * dist\_star + acceleration\_y\_constraint + speed\_y\_constraint + 2 *y\_lim\_constraint +  \frac{ratio\_fuel}{2} $$

In [ ]:
df_penalty["sum_penalty"] = 3 * df_penalty.dist_star + df_penalty.acceleration_constraint +\
      df_penalty.speed_constraint + 2 * df_penalty.y_lim_constraint +\
          df_penalty.ratio_fuel/2

fig = px.line(df_penalty.sum_penalty[:70], title = "Reward function",
        labels = {"index" : "Time", "value" : "Reward"})
fig.show()